## Superposition principle for beams

The superposition principle states that for elastic beams, the total displacement or stress at a point subjected to multiple loads is the sum of the displacements or stresses caused by each load acting independently.

<div align="center">
  <img src="Problem_image.jpeg" alt="Alt text">
</div>

Maxima script below to solve the problem using the superposition principle

```maxima

(%i2)	/* ---------------------------- */
	/* PART A: DISTRIBUTED LOAD CASE */
	/* ---------------------------- */
	
	/* 1A: Solve ODE for Segment 1 (distributed load) */
	eq: 'diff(u(z), z, 4) = p / (E * I);  /* 4th-order beam equation */
	sol_1A: desolve(eq, u(z));            /* General solution for segment 1A */
	
(eq)	'diff(u(z),z,4)=p/(E*I)
(sol_1A)	u(z)=(z^3*(at('diff(u(z),z,3),z=0)))/6+(z^2*(at('diff(u(z),z,2),z=0)))/2+z*(at('diff(u(z),z,1),z=0))+(p*z^4)/(24*E*I)+
u(0)
(%i4)	/* 2A: Solve ODE for Segment 2 (unloaded) */
	eq: 'diff(u(z), z, 4) = 0;            /* Homogeneous equation */
	sol_2A: desolve(eq, u(z));            /* General solution for segment 2A */
	
(eq)	'diff(u(z),z,4)=0

(sol_2A)	u(z)=(z^3*(at('diff(u(z),z,3),z=0)))/6+(z^2*(at('diff(u(z),z,2),z=0)))/2+z*(at('diff(u(z),z,1),z=0))+u(0)
(%i6)	/* Apply initial conditions at z=0 */
	sol_1A: rhs(subst([
	    u(0) = c_1,               /* Deflection at z=0 */
	    'diff(u(z), z, 1) = c_2,  /* Slope at z=0 */
	    'diff(u(z), z, 2) = c_3,  /* Moment at z=0 */
	    'diff(u(z), z, 3) = c_4   /* Shear at z=0 */
	], sol_1A));
	
	sol_2A: rhs(subst([
	    u(0) = c_5,               /* Deflection at z=0 */
	    'diff(u(z), z, 1) = c_6,  /* Slope at z=0 */
	    'diff(u(z), z, 2) = c_7,  /* Moment at z=0 */
	    'diff(u(z), z, 3) = c_8   /* Shear at z=0 */
	], sol_2A));
	
(sol_1A)	(p*z^4)/(24*E*I)+(c_4*z^3)/6+(c_3*z^2)/2+c_2*z+c_1
(sol_2A)	(c_8*z^3)/6+(c_7*z^2)/2+c_6*z+c_5
(%i14)	/* Interface continuity at z=Z */
	eq1: subst(Z, z, sol_1A) = subst(Z, z, sol_2A);              /* Deflection continuity */
	eq2: subst(Z, z, diff(sol_1A, z, 1)) = subst(Z, z, diff(sol_2A, z, 1));  /* Slope continuity */
	eq3: subst(Z, z, diff(sol_1A, z, 2)) = subst(Z, z, diff(sol_2A, z, 2));  /* Moment continuity */
	eq4: subst(Z, z, diff(sol_1A, z, 3)) = subst(Z, z, diff(sol_2A, z, 3));  /* Shear continuity */
	
	/* Boundary conditions at supports */
	eq5: subst(0, z, sol_1A) = 0;               /* Deflection = 0 at left support (z=0) */
	eq6: subst(L, z, sol_2A) = 0;               /* Deflection = 0 at right support (z=L) */
	eq7: subst(0, z, diff(sol_1A, z, 2)) = 0;   /* Moment = 0 at left support (z=0) */
	eq8: subst(L, z, diff(sol_2A, z, 2)) = 0;   /* Moment = 0 at right support (z=L) */
	
(eq1)	(Z^4*p)/(24*E*I)+(Z^3*c_4)/6+(Z^2*c_3)/2+Z*c_2+c_1=(Z^3*c_8)/6+(Z^2*c_7)/2+Z*c_6+c_5
(eq2)	(Z^3*p)/(6*E*I)+(Z^2*c_4)/2+Z*c_3+c_2=(Z^2*c_8)/2+Z*c_7+c_6
(eq3)	(Z^2*p)/(2*E*I)+Z*c_4+c_3=Z*c_8+c_7
(eq4)	(Z*p)/(E*I)+c_4=c_8
(eq5)	c_1=0

(eq6)	(L^3*c_8)/6+(L^2*c_7)/2+L*c_6+c_5=0

(eq7)	c_3=0

(eq8)	L*c_8+c_7=0
(%i17)	/* Solve system for integration constants */
	eq_list: [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8]$
	var_list: [c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8]$
	constants: linsolve(eq_list, var_list);      /* 8x8 linear system solution */
	
(constants)	[c_1=0,c_2=((Z^4-4*L*Z^3+4*L^2*Z^2)*p)/(24*E*I*L),c_3=0,c_4=((Z^2-2*L*Z)*p)/(2*E*I*L),c_5=-((Z^4*p)/(24*E*I)),c_6=((Z^4+4*L^2*Z^2)*p)/(24*E*I*L),c_7=-((Z^2*p)/(2*E*I)),c_8=(Z^2*p)/(2*E*I*L)]
(%i22)	/* Apply constants to solutions */
	sol_1A: subst(constants, sol_1A);            /* Final solution segment 1A */
	sol_2A: subst(constants, sol_2A);            /* Final solution segment 2A */
	
	/* ---------------------------- */
	/* PART B: POINT LOAD CASE */
	/* ---------------------------- */
	
	/* 1B: Solve ODE for both segments (unloaded) */
	eq: 'diff(u(z), z, 4) = 0;            /* Homogeneous equation */
	sol_1B: desolve(eq, u(z));            /* General solution (same for both segments) */
	sol_2B: sol_1B;                       /* Copy solution for segment 2B */
	
(sol_1A)	(p*z^4)/(24*E*I)+((Z^2-2*L*Z)*p*z^3)/(12*E*I*L)+((Z^4-4*L*Z^3+4*L^2*Z^2)*p*z)/(24*E*I*L)
(sol_2A)	(Z^2*p*z^3)/(12*E*I*L)-(Z^2*p*z^2)/(4*E*I)+((Z^4+4*L^2*Z^2)*p*z)/(24*E*I*L)-(Z^4*p)/(24*E*I)
(eq)	'diff(u(z),z,4)=0

(sol_1B)	u(z)=(z^3*(at('diff(u(z),z,3),z=0)))/6+(z^2*(at('diff(u(z),z,2),z=0)))/2+z*(at('diff(u(z),z,1),z=0))+u(0)
(sol_2B)	u(z)=(z^3*(at('diff(u(z),z,3),z=0)))/6+(z^2*(at('diff(u(z),z,2),z=0)))/2+z*(at('diff(u(z),z,1),z=0))+u(0)
(%i24)	/* Apply initial conditions at z=0 */
	sol_1B: rhs(subst([
	    u(0) = c_1,               /* Deflection at z=0 */
	    'diff(u(z), z, 1) = c_2,  /* Slope at z=0 */
	    'diff(u(z), z, 2) = c_3,  /* Moment at z=0 */
	    'diff(u(z), z, 3) = c_4   /* Shear at z=0 */
	], sol_1B));
	
	sol_2B: rhs(subst([
	    u(0) = c_5,               /* Deflection at z=0 */
	    'diff(u(z), z, 1) = c_6,  /* Slope at z=0 */
	    'diff(u(z), z, 2) = c_7,  /* Moment at z=0 */
	    'diff(u(z), z, 3) = c_8   /* Shear at z=0 */
	], sol_2B));
	
(sol_1B)	(c_4*z^3)/6+(c_3*z^2)/2+c_2*z+c_1
(sol_2B)	(c_8*z^3)/6+(c_7*z^2)/2+c_6*z+c_5
(%i32)	/* Interface conditions at z=W (point load location) */
	eq1: subst(W, z, sol_1B) = subst(W, z, sol_2B);              /* Deflection continuity */
	eq2: subst(W, z, diff(sol_1B, z, 1)) = subst(W, z, diff(sol_2B, z, 1));  /* Slope continuity */
	eq3: subst(W, z, diff(sol_1B, z, 2)) = subst(W, z, diff(sol_2B, z, 2));  /* Moment continuity */
	/* Shear force jump due to point load F */
	eq4: -E*I*subst(W, z, diff(sol_1B, z, 3)) + E*I*subst(W, z, diff(sol_2B, z, 3)) = F;
	
	/* Boundary conditions at supports (same as Part A) */
	eq5: subst(0, z, sol_1B) = 0;               /* Deflection = 0 at left support */
	eq6: subst(L, z, sol_2B) = 0;               /* Deflection = 0 at right support */
	eq7: subst(0, z, diff(sol_1B, z, 2)) = 0;   /* Moment = 0 at left support */
	eq8: subst(L, z, diff(sol_2B, z, 2)) = 0;   /* Moment = 0 at right support */
	
(eq1)	(W^3*c_4)/6+(W^2*c_3)/2+W*c_2+c_1=(W^3*c_8)/6+(W^2*c_7)/2+W*c_6+c_5
(eq2)	(W^2*c_4)/2+W*c_3+c_2=(W^2*c_8)/2+W*c_7+c_6
(eq3)	W*c_4+c_3=W*c_8+c_7
(eq4)	E*I*c_8-E*I*c_4=F

(eq5)	c_1=0

(eq6)	(L^3*c_8)/6+(L^2*c_7)/2+L*c_6+c_5=0

(eq7)	c_3=0

(eq8)	L*c_8+c_7=0
(%i35)	/* Solve system for integration constants */
	eq_list: [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8]$
	var_list: [c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8]$
	constants: linsolve(eq_list, var_list);      /* 8x8 linear system solution */
	
(constants)	[c_1=0,c_2=(F*W^3-3*F*L*W^2+2*F*L^2*W)/(6*E*I*L),c_3=0,c_4=(F*W-F*L)/(E*I*L),c_5=-((F*W^3)/(6*E*I)),
c_6=(F*W^3+2*F*L^2*W)/(6*E*I*L),c_7=-((F*W)/(E*I)),c_8=(F*W)/(E*I*L)]
(%i37)	/* Apply constants to solutions */
	sol_1B: subst(constants, sol_1B);            /* Final solution segment 1B */
	sol_2B: subst(constants, sol_2B);            /* Final solution segment 2B */
	
	/* ---------------------------- */
	/* NUMERICAL EVALUATION */
	/* ---------------------------- */
	
	/* Define physical parameters */
(sol_1B)	((F*W-F*L)*z^3)/(6*E*I*L)+((F*W^3-3*F*L*W^2+2*F*L^2*W)*z)/(6*E*I*L)
(sol_2B)	(F*W*z^3)/(6*E*I*L)-(F*W*z^2)/(2*E*I)+((F*W^3+2*F*L^2*W)*z)/(6*E*I*L)-(F*W^3)/(6*E*I)
(%i42)	parameters: [p = 3000, F = 50000, Z = 4, W = 7, L = 10];
	
	/* Substitute numerical values */
	sol_1A: ev(sol_1A, parameters);       /* Numerical solution segment 1A */
	sol_1B: ev(sol_1B, parameters);       /* Numerical solution segment 1B */
	sol_2A: ev(sol_2A, parameters);       /* Numerical solution segment 2A */
	sol_2B: ev(sol_2B, parameters);       /* Numerical solution segment 2B */
	
(parameters)	[p=3000,F=50000,Z=4,W=7,L=10]
(sol_1A)	(125*z^4)/(E*I)-(1600*z^3)/(E*I)+(51200*z)/(E*I)
(sol_1B)	(227500*z)/(E*I)-(2500*z^3)/(E*I)
(sol_2A)	(400*z^3)/(E*I)-(12000*z^2)/(E*I)+(83200*z)/(E*I)-32000/(E*I)
(sol_2B)	(17500*z^3)/(3*E*I)-(175000*z^2)/(E*I)+(1452500*z)/(E*I)-8575000/(3*E*I)
(%i45)	/* Post-processing */
	deflection_at_7_1: ev(sol_1B + sol_2A, z = 7);              /* Total deflection at z=7 */
	deflection_at_7_2: ev(sol_2A + sol_2B, z = 7);              /* Total deflection at z=7 */
	slope_total: subst(0, z, -diff(sol_1A + sol_1B, z, 1));   /*  Slope at z=0 */
(deflection_at_7_1)	834600/(E*I)
(deflection_at_7_2)	834600/(E*I)
(slope_total)	-(278700/(E*I))


```